# C++



El programa crea un árbol de procesos, el proceso A crea tres procesos hijos, llamados B, C y D, el proceso hijo B crea otros dos procesos hijos, E y F, y el proceso D crea un último proceso hijo llamado G. Se le agregó tiempos de retraso para poder monitorear con el comando ps la correcta creación de los procesos.

In [ ]:
%%writefile Fork.cpp
#include <iostream>
#include <unistd.h>
#include <sys/wait.h>

using namespace std;

void print_process_message(char char_of_process)
{
    cout << "Proceso " << char_of_process << " (PID: " << getpid() << " | PPID: " << getppid() << " )" << endl;
}

int main(int argc, char *argv[])
{
    pid_t pidA, pidB, pidC, pidD, pidE, pidF, pidG;
    pidA = getpid();
    print_process_message('A');
    pidB = fork();
    if (pidB == 0)
    {
        pidB = getpid();
        print_process_message('B');
        pidE = fork();
        if (pidE == 0)
        {
            pidE = getpid();
            print_process_message('E');
            sleep(30);
            exit(0);
        }
        pidF = fork();
        if (pidF == 0)
        {
            pidF = getpid();
            print_process_message('F');
            sleep(30);
            exit(0);
        }
        sleep(30);
        wait(NULL);
        wait(NULL);
        exit(0);
    }
    pidC = fork();
    if (pidC == 0)
    {
        pidC = getpid();
        print_process_message('C');
        sleep(30);
        exit(0);
    }
    pidD = fork();
    if (pidD == 0)
    {
        pidD = getpid();
        print_process_message('D');
        pidG = fork();
        if (pidG == 0)
        {
            pidG = getpid();
            print_process_message('G');
            sleep(30);
            exit(0);
        }
        sleep(30);
        wait(NULL);
        exit(0);
    }
    sleep(30);
    wait(NULL);
    wait(NULL);
    wait(NULL);
    return 0;
}

Writing Fork.cpp


Verificamos que existe el archivo Fork.cpp

In [ ]:
!ls -l

total 8
-rw-r--r-- 1 root root 1669 Apr 11 18:33 Fork.cpp
drwxr-xr-x 1 root root 4096 Apr 10 13:31 sample_data


Compilamos el archivo Fork.cpp y generamos el binario.

In [ ]:
%%shell
g++ Fork.cpp -o Fork.bin

Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaC"

In [ ]:
%%shell
nohup ./Fork.bin 1>salidaC 2>/dev/null & 

Mostramos la salida del programa. 

In [ ]:
%%shell
g++ -o Fork Fork.cpp
./Fork

Proceso A con PID 4965, mi padre es: 954
Proceso B con PID 4971, mi padre es: 4965
Proceso C con PID 4972, mi padre es: 4965
Proceso D con PID 4974, mi padre es: 4965
Proceso F con PID 4975, mi padre es: 4971
Proceso E con PID 4973, mi padre es: 4971
Proceso G con ID 4976, mi padre es: 4974


Visualizamos los procesos generados por este programa

In [ ]:
%%shell
ps -ef | grep Fork.bin

root        1570       1  0 18:35 ?        00:00:00 ./Fork.bin
root        1571    1570  0 18:35 ?        00:00:00 ./Fork.bin
root        1572    1570  0 18:35 ?        00:00:00 ./Fork.bin
root        1573    1570  0 18:35 ?        00:00:00 ./Fork.bin
root        1574    1573  0 18:35 ?        00:00:00 ./Fork.bin
root        1575    1571  0 18:35 ?        00:00:00 ./Fork.bin
root        1576    1571  0 18:35 ?        00:00:00 ./Fork.bin
root        1591     954  0 18:35 ?        00:00:00 /bin/bash -c ps -ef | grep Fork.bin 
root        1593    1591  0 18:35 ?        00:00:00 grep Fork.bin


Ahora visualizamos el arbol de procesos, ejecutando el siguiente comando:
***pstree -pc PID***

**PID**: PID del proceso A (Lo pueden encontrar dentro del archivo "salidaC")

In [ ]:
%%shell
pstree -pc 1570

Fork.bin(1570)─┬─Fork.bin(1571)─┬─Fork.bin(1575)
               │                └─Fork.bin(1576)
               ├─Fork.bin(1572)
               └─Fork.bin(1573)───Fork.bin(1574)



# Java




El programa recibe 2 argumentos, la cantidad de hijos que debe generar el proceso al ejecutarse, y el nivel de jerarquia que este tiene. Para cumplimentar el ejercicio tiene un corte cuando alcanzan el nivel 2.

In [ ]:
%%writefile ProcessTree.java
package processTree;

import java.io.IOException;

public class ProcessTree {
	
	public static void main(String[] args) throws IOException, InterruptedException {
		
		ProcessHandle process = ProcessHandle.current();
		long pid = process.pid();
		long ppid = process.parent().get().pid();
		System.out.println("Soy el proceso: "+pid+", Mi papa es: "+ppid);
		
		int level = Integer.parseInt(args[1]);
		if(level==2) {
			Thread.sleep(10000);
			return;
		}	
		
		int childrenNum = Integer.parseInt(args[0]);
		
		Process childs[] = new Process[childrenNum];
		
		for (int i = 1; i <= childrenNum; i++) {
			ProcessBuilder pb = new ProcessBuilder("java","ProcessTree.java",String.valueOf(childrenNum-i),String.valueOf(level+1));
			pb.inheritIO();
			childs[i-1] = pb.start();
		}
		Thread.sleep(10000);
		for (int i = 0; i < childs.length; i++) {
			childs[i].waitFor();
		}
		
	}
	
}

Writing ProcessTree.java


Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaJava"

In [ ]:
!nohup java ProcessTree.java 3 0 1>salidaJava 2>/dev/null &

Mostramos la salida del programa

In [ ]:
!java ProcessTree.java 3 0

Soy el proceso: 7253, Mi papa es: 954
Soy el proceso: 7285, Mi papa es: 7253
Soy el proceso: 7275, Mi papa es: 7253
Soy el proceso: 7290, Mi papa es: 7253
Soy el proceso: 7365, Mi papa es: 7275
Soy el proceso: 7357, Mi papa es: 7285
Soy el proceso: 7369, Mi papa es: 7275


Vemos el arbol de procesos que genero este programa.

***IMPORTANTE:*** Utilizar el PID del primer proceso que sale dentro del archivo salidaJava.

*NO UTILIZAR EL PID DEL EJEMPLO DE ARRIBA PORQUE NO DEJA CORRER HASTA QUE NO TERMINEN TODOS LOS PROCESOS*

In [ ]:
!pstree -pT 7253


# Python



El programa crea un árbol de procesos, el proceso A crea tres procesos hijos, llamados B, C y D, el proceso hijo B crea otros dos procesos hijos, E y F, y el proceso D crea un último proceso hijo llamado G. Se le agregó tiempos de retraso para poder monitorear con el comando ps la correcta creación de los procesos.

In [2]:

import os
import sys
import time

def print_process_message(char_of_process):
    print("Soy el proceso " + char_of_process + " ( PID: " + str(os.getpid()) + " | PPID: " + str(os.getppid()) + " )")

	
def parent():

    print_process_message('A')
    pidB = os.fork()

    if pidB == 0:
        pidB = os.getpid()
        print_process_message('B')
        pidE = os.fork()
        if pidE == 0:
            pidE = os.getpid()
            print_process_message('E')
            time.sleep(30)
            exit(0)
        
        pidF = os.fork()
        if pidF == 0:
            pidF = os.getpid()
            print_process_message('F')
            time.sleep(30)
            exit(0)
        
        time.sleep(30)
        exit(0)
    
    pidC = os.fork()
    if pidC == 0:
        pidC = os.getpid()
        print_process_message('C')
        time.sleep(30)
        exit(0)
    
    pidD = os.fork()
    if pidD == 0:
        pidD = os.getpid()
        print_process_message('D')
        pidG = os.fork()
        if pidG == 0:
            pidG = os.getpid()
            print_process_message('G')
            time.sleep(30)
            exit(0)
        
        time.sleep(30)
        exit(0)
    
    time.sleep(30)
    return 0
    
    

parent()

Proceso A con PID 8875, mi padre es: 76
Proceso B con PID 31023, mi padre es: 8875Proceso A con PID 8875, mi padre es: 76
Proceso D con PID 31027, mi padre es: 8875

Proceso A con PID 8875, mi padre es: 76
Proceso C con PID 31024, mi padre es: 8875Proceso F con PID 31035, mi padre es: 31023Proceso E con PID 31034, mi padre es: 31023


Proceso G con ID 31038, mi padre es: 31027
Proceso A con PID 8875, mi padre es: 76


0

Proceso C con PID 31181, mi padre es: 31023
Proceso D con PID 31182, mi padre es: 31023
Proceso G con ID 31191, mi padre es: 31182Proceso D con PID 31194, mi padre es: 31024

Proceso F con PID 31199, mi padre es: 31034
Proceso G con ID 31204, mi padre es: 31194


Visualizamos el arbol de procesos que genera este programa.

In [1]:
!pstree -pc 8875